## Harjoitustyö



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [3]:
!pip install scrapy

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached Scrapy-2.5.0-py2.py3-none-any.whl (254 kB)
Processing /root/.cache/pip/wheels/dc/d0/bf/0cc715c01fce0bace63b46283acf5cc630d5e5dbb4602c54e5/PyDispatcher-2.0.5-py3-none-any.whl
  Using cached zope.interface-5.4.0-cp37-cp37m-manylinux2010_x86_64.whl (251 kB)
Processing /root/.cache/pip/wheels/ca/44/01/3592ccfbcfaee4ab297c4097e6e9dbe1c7697e3531a39877ab/Protego-0.1.16-py3-none-any.whl
  Using cached w3lib-1.22.0-py2.py3-none-any.whl (20 kB)
  Using cached service_identity-18.1.0-py2.py3-none-any.whl (11 kB)
  Using cached itemadapter-0.2.0-py3-none-any.whl (9.3 kB)
  Using cac

In [22]:
!scrapy genspider etuovi_scraper etuovi.com

Created spider 'etuovi_scraper' using template 'basic' 


In [23]:
## lopullinen raapija

import scrapy
# raapija, joka kerää etuovi.com:sta ilmoituksien tiedoista
# asuntotyypin, osoitteen, hinnan, valmistuvuoden ja pinta-alan.
# (Kyseessä Tampereen omakotitalot)
class EtuoviScraperSpider(scrapy.Spider):
    name = 'etuovi_scraper'
    allowed_domains = ['etuovi.com']
    # assing a page url below
    start_urls = \
    ['https://www.etuovi.com/myytavat-asunnot?haku=M1644983732&sivu=0']

  # Funkito parse(self, response) käy läpi aloitusosoitteesta eteenpäin
  # sivuja, joilta dataa kerätään. Parse kutsuu parse_page-funktiota, jossa
  # on määritelty varsinainen kerättävä tieto.
  # max 5 sivua tässä esimerkissä
    def parse(self, response):
        for i in range(1, 21):
            url=response.request.url[:-1]+str(i)
            yield scrapy.Request(url, callback=self.parse_page)
        
    
    def parse_page(self, response):
        description_texts = response.css(\
          '[class="flexboxgrid__col-xs-12__1I1LS"] *::text').extract()
        size_text = response.css(\
          '[class="flexboxgrid__col-xs__26GXk flexboxgrid__col-md-4__2DYW-"] > span::text').extract()
        year_text = response.css(\
          '[class="flexboxgrid__col-xs-3__3Kf8r flexboxgrid__col-md-4__2DYW-"] > span::text'\
          ).extract()
        price_text = response.css(\
          '[class="flexboxgrid__col-xs-4__p2Lev flexboxgrid__col-md-4__2DYW-"] > span::text'\
          ).extract()

        for i in range(len(price_text)):
            yield {
                'house_type' : description_texts[2*i],
                'address': description_texts[2*i+1],
                'price': price_text[i],
                'year': year_text[i],
                'size': size_text[i]
            }

In [24]:
!scrapy runspider etuovi_scraper.py -o out.json

2021-04-23 15:18:27 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-04-23 15:18:27 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.7.10 (default, Feb 26 2021, 18:47:35) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.4.7, Platform Linux-4.14.225-169.362.amzn2.x86_64-x86_64-with-debian-10.6
2021-04-23 15:18:27 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-04-23 15:18:27 [scrapy.crawler] INFO: Overridden settings:
{'SPIDER_LOADER_WARN_ONLY': True}
2021-04-23 15:18:27 [scrapy.extensions.telnet] INFO: Telnet Password: c610491975b07e21
2021-04-23 15:18:27 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
202

In [3]:
!git commit -m "data: Tampere, Turku, Jyväskylä, Oulu"

[master 40097a5] spider
 1 file changed, 305 insertions(+)
 create mode 100644 joda_ht.ipynb


In [2]:
!git config --global user.email "sara.paukkeri@hotmail.com"

## Datan siivoaminen ja piirteiden  muokkaus

In [ ]:
import json

In [ ]:
# read out.json file to d
d = open('out.json')
data = json.load(d)
d.close()
data[0:1]

In [ ]:
# convert array to dataframe
df = pd.DataFrame(data)
df

In [ ]:
# split column size to size and living_space and remove "m2" and spaces
# add rows with empty values(="-") to remove
remove=[]

living_space=[]
for i in range(0, df['size'].size):
  if "/" in df['size'][i]: # size is "-" or only one size available
    sizes = df['size'][i].split("/")
    df['size'][i] = sizes[1].replace(" ", "").replace("m\u00b2","").replace(",",".")
    living_space.append(sizes[0].replace(" ", "").replace(",","."))
  else: 
    living_space.append(0)
    remove.append(i)
    df['size'][i]=0
  
df['living_space']=living_space

df['size'] = df['size'].astype(float)
df.living_space = df.living_space.astype(float)

In [ ]:
# remove € from price and set datatype to int
for i in range(0, df.price.size): #=0; i < df.price.length; i++):
  if df.price[i]=="-" and i not in remove:
    remove.append(i)
  else: 
    df.price[i] = df.price[i].replace("\xa0", "").replace('€',"")

df.price = df.price.astype(int)

In [ ]:
df.head()

In [ ]:
# Split address to address, region, area
area=[]
city=[]

for i in range(0, df['address'].size):
  add = df['address'][i].split(", ")
  df['address'][i] = add[0]
  area.append(add[1])
  city.append(add[2])
  
df['area']=area
df['city'] = city

In [ ]:
rooms=[]

for i in range(0, df['house_type'].size):
  house = df['house_type'][i].split(" | ")
  df['house_type'][i] = house[0]
  rooms.append(house[1])
  
df['rooms']=rooms

In [ ]:
for i in range(0, df.year.size):
  if df.year[i]=="-" and i not in remove:
    remove.append(i)

df = df.drop(remove)

df['year'] = df['year'].astype(int)

In [ ]:
df.head()

In [ ]:
df['geo_address'] = df["address"]+ ", " + df["city"] + ", Finland"
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent = "joda ht")

In [ ]:
df["loc"] = df["geo_address"].apply(geolocator.geocode)
# select koordinates from loc
df["point"] = df["loc"].apply(lambda loc: tuple(loc.point) if loc else None)

df[['lat', 'lon', 'altitude']] = pd.DataFrame(df['point'].to_list(), index=df.index)

In [ ]:
df.head()

In [ ]:
df['loc'][0]

###Datan kuvaileminen

In [ ]:
fig = plt.figure(constrained_layout=True)
gs = fig.add_gridspec(2, 2)
fig.suptitle('Muuttujien jakaumat')

f_ax1 = fig.add_subplot(gs[0, 0])
f_ax1.hist(df['price'])
f_ax1.set_title('Price')
f_ax2 = fig.add_subplot(gs[0, 1])
f_ax2.hist(df['year'])
f_ax2.set_title('Year')
f_ax3 = fig.add_subplot(gs[1, 0])
f_ax3.hist(df['size'])
f_ax3.set_title('Size')
f_ax4 = fig.add_subplot(gs[1, 1])
f_ax4.hist(df['living_space'])
f_ax4.set_title('Living Space')

In [ ]:
from collections import Counter

value = Counter(df['house_type']).values()
label = Counter(df['house_type']).keys()

plt.pie(value, labels = label, autopct='%1.1f%%')
plt.show()

In [ ]:
from pandas.plotting import scatter_matrix

df_int = df.loc[:,['price', 'year', 'living_space', 'size']]

scatter_matrix(df_int, figsize=(12,12), diagonal="kde");

In [ ]:
area_value = Counter(df['city']).values()
area_label = Counter(df['city']).keys()

plt.pie(area_value, labels = area_label, autopct='%1.1f%%')
plt.show()

In [ ]:
import folium
from folium.plugins import MarkerCluster

#Define coordinates of where we want to center our map
center = [61.6027417, 23.8721711]

#Create the map
my_map = folium.Map(location = center, zoom_start = 5)

# Add addresses to the map
for index, row in df.iterrows():
  if row.point != None:
    text ="Price: " + str(row.price) + " €" + "\n Size: " + str(row['size']) + " m2 " + "Address: " + row.address
    folium.Marker([row.lat,row.lon], popup = text).add_to(my_map)

#Display the map
my_map